<a href="https://colab.research.google.com/github/mabdulhadi/360monodepth/blob/main/Session2/UnsolvedNotebooks/W_1_3_1_function_calling_Structured_Outputs_langchain_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structured Outputs
Guarantee model responses strictly conform to your JSON schema for reliable, type-safe data structures.

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
groq_key = userdata.get('groqdummy')

In [ ]:
from groq import Groq
from pydantic import BaseModel
from typing import Literal
import json


client = Groq(api_key = groq_key)
#Getting a structured response from unstructured text
class ProductReview(BaseModel):
    product_name: str
    rating: float
    sentiment: Literal["positive", "negative", "neutral"]
    key_features: list[str]

response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=[
        {"role": "system", "content": "Extract product review information from the text."},
        {
            "role": "user",
            "content": "I bought the UltraSound Headphones last week and I'm really impressed! The noise cancellation is amazing and the battery lasts all day. Sound quality is crisp and clear. I'd give it 4.5 out of 5 stars.",
        },
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "product_review",
            "schema": ProductReview.model_json_schema()
        }
    }
)

review = ProductReview.model_validate(json.loads(response.choices[0].message.content))
print(json.dumps(review.model_dump(), indent=2))

{
  "product_name": "UltraSound Headphones",
  "rating": 4.5,
  "sentiment": "positive",
  "key_features": [
    "amazing noise cancellation",
    "all-day battery life",
    "crisp and clear sound quality"
  ]
}


# Email Classification

In [ ]:
from groq import Groq
from pydantic import BaseModel
import json

#client = Groq()

class KeyEntity(BaseModel):
    entity: str
    type: str
#You can classify emails into structured categories with confidence scores, priority levels, and suggested actions.
class EmailClassification(BaseModel):
    category: str
    priority: str
    confidence_score: float
    sentiment: str
    key_entities: list[KeyEntity]
    suggested_actions: list[str]
    requires_immediate_attention: bool
    estimated_response_time: str

response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are an email classification expert. Classify emails into structured categories with confidence scores, priority levels, and suggested actions.",
        },
        {"role": "user", "content": "Subject: URGENT: Server downtime affecting production\\n\\nHi Team,\\n\\nOur main production server went down at 2:30 PM EST. Customer-facing services are currently unavailable. We need immediate action to restore services. Please join the emergency call.\\n\\nBest regards,\\nDevOps Team"},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "email_classification",
            "schema": EmailClassification.model_json_schema()
        }
    }
)

email_classification = EmailClassification.model_validate(json.loads(response.choices[0].message.content))
print(json.dumps(email_classification.model_dump(), indent=2))

{
  "category": "Incident Report",
  "priority": "Critical",
  "confidence_score": 0.97,
  "sentiment": "Negative",
  "key_entities": [
    {
      "entity": "production server",
      "type": "Infrastructure"
    },
    {
      "entity": "2:30 PM EST",
      "type": "Time"
    },
    {
      "entity": "customer-facing services",
      "type": "Service"
    },
    {
      "entity": "DevOps Team",
      "type": "Team"
    }
  ],
  "suggested_actions": [
    "Join the emergency call immediately",
    "Assess server status and initiate recovery procedures",
    "Communicate ETA updates to stakeholders",
    "Post-incident review and root-cause analysis"
  ],
  "requires_immediate_attention": true,
  "estimated_response_time": "<15 minutes"
}


# Support Ticket Classification

In [ ]:
from groq import Groq
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from enum import Enum
import json

#client = Groq()

class SupportCategory(str, Enum):
    API = "api"
    BILLING = "billing"
    ACCOUNT = "account"
    BUG = "bug"
    FEATURE_REQUEST = "feature_request"
    INTEGRATION = "integration"
    SECURITY = "security"
    PERFORMANCE = "performance"

class Priority(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class CustomerTier(str, Enum):
    FREE = "free"
    PAID = "paid"
    ENTERPRISE = "enterprise"
    TRIAL = "trial"

class CustomerInfo(BaseModel):
    name: str
    company: Optional[str] = None
    tier: CustomerTier

class TechnicalDetail(BaseModel):
    component: str
    error_code: Optional[str] = None
    description: str

class SupportTicket(BaseModel):
    category: SupportCategory
    priority: Priority
    urgency_score: float
    customer_info: CustomerInfo
    technical_details: List[TechnicalDetail]
    keywords: List[str]
    requires_escalation: bool
    estimated_resolution_hours: float
    follow_up_date: Optional[str] = Field(None, description="ISO datetime string")
    summary: str

response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=[
        {
            "role": "system",
            "content": """You are a customer support ticket classifier for SaaS companies.
                         Analyze support tickets and categorize them for efficient routing and resolution.
                         Output JSON only using the schema provided.""",
        },
        {
            "role": "user",
            "content": """Hello! I love your product and have been using it for 6 months.
                         I was wondering if you could add a dark mode feature to the dashboard?
                         Many of our team members work late hours and would really appreciate this.
                         Also, it would be great to have keyboard shortcuts for common actions.
                         Not urgent, but would be a nice enhancement!
                         Best, Mike from StartupXYZ"""
        },
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "support_ticket_classification",
            "schema": SupportTicket.model_json_schema()
        }
    }
)

raw_result = json.loads(response.choices[0].message.content or "{}")
result = SupportTicket.model_validate(raw_result)
print(result.model_dump_json(indent=2))

{
  "category": "feature_request",
  "priority": "low",
  "urgency_score": 2.0,
  "customer_info": {
    "name": "Mike",
    "company": "StartupXYZ",
    "tier": "paid"
  },
  "technical_details": [
    {
      "component": "dashboard",
      "error_code": null,
      "description": "Request to add dark mode theme support for the dashboard UI"
    },
    {
      "component": "keyboard_shortcuts",
      "error_code": null,
      "description": "Request to add keyboard shortcuts for common actions"
    }
  ],
  "keywords": [
    "dark_mode",
    "dashboard",
    "keyboard_shortcuts",
    "enhancement",
    "UI"
  ],
  "requires_escalation": false,
  "estimated_resolution_hours": 120.0,
  "follow_up_date": null,
  "summary": "Customer requesting dark mode for dashboard and keyboard shortcuts for common actions. Non-urgent enhancement request from 6-month user."
}


In [ ]:
def koi_fun(input_variables,two, two,):


  #body functionality  API, calcualtion, data loader

  return

SyntaxError: incomplete input (ipython-input-1653422645.py, line 1)

# How to do tool/function calling


We use the term tool calling interchangeably with function calling. Although
function calling is sometimes meant to refer to invocations of a single function,
we treat all models as though they can return multiple tool or function calls in
each message.
:::

Tool calling allows a model to respond to a given prompt by generating output that
matches a user-defined schema. While the name implies that the model is performing
some action, this is actually not the case! The model is coming up with the
arguments to a tool, and actually running the tool (or not) is up to the user -
for example, if you want to [extract output matching some schema](/docs/tutorials/extraction)
from unstructured text, you could give the model an "extraction" tool that takes
parameters matching the desired schema, then treat the generated output as your final
result.

A tool call includes a name, arguments dict, and an optional identifier. The
arguments dict is structured `{argument_name: argument_value}`.

Many LLM providers, including [Anthropic](https://www.anthropic.com/),
[Cohere](https://cohere.com/), [Google](https://cloud.google.com/vertex-ai),
[Mistral](https://mistral.ai/), [OpenAI](https://openai.com/), and others,
support variants of a tool calling feature. These features typically allow requests
to the LLM to include available tools and their schemas, and for responses to include
calls to these tools. For instance, given a search engine tool, an LLM might handle a
query by first issuing a call to the search engine. The system calling the LLM can
receive the tool call, execute it, and return the output to the LLM to inform its
response. LangChain includes a suite of [built-in tools](/docs/integrations/tools/)
and supports several methods for defining your own [custom tools](/docs/how_to/custom_tools).
Tool-calling is extremely useful for building [tool-using chains and agents](/docs/how_to#tools),
and for getting structured outputs from models more generally.

Providers adopt different conventions for formatting tool schemas and tool calls.
For instance, Anthropic returns tool calls as parsed structures within a larger content block:
```python
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```
whereas OpenAI separates tool calls into a distinct parameter, with arguments as JSON strings:
```python
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```
LangChain implements standard interfaces for defining tools, passing them to LLMs,
and representing tool calls.

## Passing tools to LLMs

Chat models supporting tool calling features implement a `.bind_tools` method, which
receives a list of LangChain [tool objects](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool)
and binds them to the chat model in its expected format. Subsequent invocations of the
chat model will include tool schemas in its calls to the LLM.

For example, we can define the schema for custom tools using the `@tool` decorator
on Python functions:

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

Or below, we define the schema using Pydantic:


In [ ]:
from pydantic import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

We can bind them to chat models as follows:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
  overrideParams={{fireworks: {model: "accounts/fireworks/models/firefunction-v1", kwargs: "temperature=0"}}}
/>

We can use the `bind_tools()` method to handle converting
`Multiply` to a "tool" and binding it to the model (i.e.,
passing it in each time the model is invoked).

In [ ]:
# # | echo: false
# # | output: false

# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
!pip install -qU "langchain[groq]"

In [ ]:
# from google.colab import userdata
# key = userdata.get('groqdummy')

In [ ]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = groq_key

from langchain.chat_models import init_chat_model

llm = init_chat_model("moonshotai/kimi-k2-instruct", model_provider="groq")

In [ ]:
llm_with_tools = llm.bind_tools(tools)

## Tool calls

If tool calls are included in a LLM response, they are attached to the corresponding
[message](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage)
or [message chunk](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk)
as a list of [tool call](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolCall.html#langchain_core.messages.tool.ToolCall)
objects in the `.tool_calls` attribute. A `ToolCall` is a typed dict that includes a
tool name, dict of argument values, and (optionally) an identifier. Messages with no
tool calls default to an empty list for this attribute.

Example:

In [ ]:
query = "what is 2*2 and 3 + 5 and 5 + 6 ?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 2, 'b': 2},
  'id': 'functions.Multiply:0',
  'type': 'tool_call'},
 {'name': 'Add',
  'args': {'a': 3, 'b': 5},
  'id': 'functions.Add:1',
  'type': 'tool_call'},
 {'name': 'Add',
  'args': {'a': 5, 'b': 6},
  'id': 'functions.Add:2',
  'type': 'tool_call'}]

The `.tool_calls` attribute should contain valid tool calls. Note that on occasion,
model providers may output malformed tool calls (e.g., arguments that are not
valid JSON). When parsing fails in these cases, instances
of [InvalidToolCall](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.InvalidToolCall.html#langchain_core.messages.tool.InvalidToolCall)
are populated in the `.invalid_tool_calls` attribute. An `InvalidToolCall` can have
a name, string arguments, identifier, and error message.

If desired, [output parsers](/docs/how_to#output-parsers) can further
process the output. For example, we can convert back to the original Pydantic class:

In [ ]:
# from langchain_core.output_parsers.openai_tools import PydanticToolsParser

# chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
# chain.invoke(query)

KeyError: 'multiply'

## Passing tool outputs to model

If we're using the model-generated tool invocations to actually call tools and want to pass the tool results back to the model, we can do so using `ToolMessage`s.

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
query = "what is 4 multiply 5"
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
print(ai_msg)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages[-1]

content="I'll multiply 4 and 5 for you." additional_kwargs={'tool_calls': [{'id': 'functions.Multiply:0', 'function': {'arguments': '{"a":4,"b":5}', 'name': 'Multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 197, 'total_tokens': 230, 'completion_time': 0.074226871, 'prompt_time': 2.6051005160000003, 'queue_time': 0.115586136, 'total_time': 2.679327387}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_b8565bb333', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--70950d4b-a450-46aa-a0ca-325e7af9210f-0' tool_calls=[{'name': 'Multiply', 'args': {'a': 4, 'b': 5}, 'id': 'functions.Multiply:0', 'type': 'tool_call'}] usage_metadata={'input_tokens': 197, 'output_tokens': 33, 'total_tokens': 230}


ToolMessage(content='20', tool_call_id='functions.Multiply:0')

In [ ]:
messages[-1].content

'2000'

In [ ]:
#llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_k4t2', 'function': {'arguments': '{"a":2000,"b":0}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 325, 'total_tokens': 345, 'completion_time': 0.037910674, 'prompt_time': 0.017878976, 'queue_time': 0.209801022, 'total_time': 0.05578965}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2d7a0e98-503e-4987-81c0-edf85b43ab8a-0', tool_calls=[{'name': 'Add', 'args': {'a': 2000, 'b': 0}, 'id': 'call_k4t2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 325, 'output_tokens': 20, 'total_tokens': 345})